In [27]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# 设置字体为Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
# 设置字体大小为14
plt.rcParams['font.size'] = 13
plt.rcParams['figure.dpi'] = 720

# 原始数据（变化率，已计算）
Year = [2018, 2019, 2020, 2021, 2022, 2023]
Carbon_Storage_Change = [-2.64, -0.20, -0.14, 0.07, 2.38, -2.79]
Water_Areas_Change = [-20.83, 0.00, -5.26, -5.56, 34.12, 89.91]
Trees_Change = [-4.50, -0.16, 0.13, 0.58, 4.72, -5.49]
Flooded_Vegetation_Change = [-35.06, 4.91, -77.78, -92.11, 366.67, 57.14]
Crops_Change = [-4.66, -12.17, 52.82, -80.70, -59.13, 66.37]
Built_Areas_Change = [6.43, 1.07, 1.59, 1.33, -2.68, 0.58]
Bare_Ground_Change = [-79.18, -65.15, -100.00, None, 240.91, 217.33]
Rangelands_Change = [24.29, 2.18, -11.75, 20.94, -16.65, 27.59]

# 归一化函数
def normalize(data):
    valid_data = [x for x in data if x is not None]
    min_val, max_val = min(valid_data), max(valid_data)
    return [(x - min_val) / (max_val - min_val) if x is not None else None for x in data]

# 归一化所有数据
x_norm = normalize(Carbon_Storage_Change)
y_data = {
    'Water Areas': normalize(Water_Areas_Change),
    'Trees': normalize(Trees_Change),
    'Flooded Vegetation Areas': normalize(Flooded_Vegetation_Change),
    'Crops': normalize(Crops_Change),
    'Built Areas': normalize(Built_Areas_Change),
    'Bare Groud Areas': normalize(Bare_Ground_Change),
    'Rangelands Areas': normalize(Rangelands_Change)
}

# 创建7个子图
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for i, (title, y_norm) in enumerate(y_data.items()):
    ax = axes[i]
    valid_x = np.array([x for x, y in zip(x_norm, y_norm) if y is not None])
    valid_y = np.array([y for y in y_norm if y is not None])
    
    # 计算指标
    r2 = r2_score(valid_y, valid_x)
    rmse = np.sqrt(mean_squared_error(valid_y, valid_x))
    mae = mean_absolute_error(valid_y, valid_x)
    
    # 绘制散点图（蓝色）
    ax.scatter(valid_x, valid_y, color='blue', alpha=0.6, label='Data')
    
    # 获取当前坐标轴范围
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    plot_min = min(x_min, y_min)
    plot_max = max(x_max, y_max)
    
    # y=x参考线（灰色虚线）扩展到整个图形范围
    ax.plot([plot_min, plot_max], [plot_min, plot_max], '--', color='grey', label='y=x')

    # 添加灰色透明缓冲带（y=x±RMSE）
    ax.fill_between([plot_min, plot_max], 
                   [plot_min-rmse, plot_max-rmse], 
                   [plot_min+rmse, plot_max+rmse], 
                   color='gray', alpha=0.2, label='±RMSE band')
    
    # 重置坐标轴范围以包含所有元素
    ax.set_xlim(plot_min, plot_max)
    ax.set_ylim(plot_min, plot_max)
    
    # 添加指标文本
    text_box = f"R2 = {r2:.3f}\nRMSE = {rmse:.3f}\nMAE = {mae:.3f}"
    ax.text(0.14, 0.95, text_box, transform=ax.transAxes,
            verticalalignment='top', bbox=dict(facecolor='white', alpha=0.7),
            fontsize=11)
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Normalized Carbon Storage Change', fontsize=14)
    ax.set_ylabel('Normalized LULC Change Rate', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11, loc='lower right', bbox_to_anchor=(0.94, 0.01))

# 隐藏多余子图
for j in range(len(y_data), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()